In [1]:
import torch
from torch_geometric.utils import from_dgl, to_networkx, k_hop_subgraph
import random
from torch_geometric.data import Data
from torch_geometric.utils import subgraph, to_undirected
from torch_cluster import random_walk
from utils import *

/home/allenwang/miniconda3/envs/gad_v2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
local_subgraph = torch.load('local_subgraphs/reddit_0.pt')
a = local_subgraph[1].node_mapping
a

tensor([  21,   49,   60,  163,  170,  257,  433,  471,  497,  517,  588,  616,
         723,  725,  841,  872, 1002, 1053, 1096, 1109, 1131, 1215, 1235, 1237,
        1255, 1310, 1430, 1444, 1493, 1527, 1528, 1683, 1775, 1790, 1986, 2134,
        2149, 2227, 2348, 2363, 2368, 2383, 2584, 2602, 2642, 2655, 2717, 2743,
        2747, 2774, 2983, 2989, 3088, 3122, 3189, 3197, 3419, 3585, 3693, 3770,
        3788, 3794, 3813, 4004, 4067, 4108, 4132, 4135, 4184, 4243, 4343, 4379,
        4384, 4395, 4436, 4447, 4677, 4909, 5127, 5218, 5330, 5422, 5478, 5547,
        5550, 5677, 5701, 5824, 5898, 5930, 5994, 6008, 6015, 6078, 6110, 6223,
        6353, 6402, 6405, 6424, 6471, 6576, 6701, 6760, 6829, 7013, 7043, 7090,
        7125, 7178, 7182, 7297, 7316, 7360, 7452, 7583, 7629, 7637, 7690, 7741,
        7760, 7855, 7882, 7939, 7999, 8004, 8035, 8053, 8066, 8157, 8190, 8206,
        8245, 8320, 8444, 8475, 8648, 8912, 9060, 9129, 9131, 9169, 9201, 9288,
        9317, 9339, 9394, 9423, 9661, 99

In [34]:
# put local subgraph to pyg dataloader
from torch_geometric.loader import DataLoader

dataloader = DataLoader(local_subgraph, batch_size=5, shuffle=False)

# iterate over the dataloader
for i, batch in enumerate(dataloader):
    print(i, batch)

0 DataBatch(x=[750], edge_index=[2, 2134], extra_x=[750, 64], num_nodes=750, node_mapping=[750], center_node_idx=[5], batch=[750], ptr=[6])
1 DataBatch(x=[668], edge_index=[2, 3172], extra_x=[668, 64], num_nodes=668, node_mapping=[668], center_node_idx=[5], batch=[668], ptr=[6])
2 DataBatch(x=[750], edge_index=[2, 3924], extra_x=[750, 64], num_nodes=750, node_mapping=[750], center_node_idx=[5], batch=[750], ptr=[6])
3 DataBatch(x=[750], edge_index=[2, 4968], extra_x=[750, 64], num_nodes=750, node_mapping=[750], center_node_idx=[5], batch=[750], ptr=[6])
4 DataBatch(x=[750], edge_index=[2, 3466], extra_x=[750, 64], num_nodes=750, node_mapping=[750], center_node_idx=[5], batch=[750], ptr=[6])
5 DataBatch(x=[750], edge_index=[2, 3318], extra_x=[750, 64], num_nodes=750, node_mapping=[750], center_node_idx=[5], batch=[750], ptr=[6])
6 DataBatch(x=[750], edge_index=[2, 3490], extra_x=[750, 64], num_nodes=750, node_mapping=[750], center_node_idx=[5], batch=[750], ptr=[6])
7 DataBatch(x=[750],

## Calling Diffusion Model

In [8]:
import torch
aug_data = torch.load('augment_samples.pt')

aug_data[0]

Data(x=[150, 2], edge_index=[2, 1672])

In [9]:
import torch

def remap_edge_index(edge_index, node_mapping):
    # Map the indices according to the node_mapping
    remapped_edge_index = node_mapping[edge_index]
    return remapped_edge_index

# Example usage
num_nodes = 5  # Adjust this according to your actual number of nodes
num_edges = 4  # Adjust this according to your actual number of edges

# Example edge_index
edge_index = torch.tensor([[0, 1, 2, 3],
                           [1, 2, 3, 4]])

# Example node_mapping (e.g., node 0 -> 2, node 1 -> 0, node 2 -> 1, etc.)
node_mapping = torch.tensor([2, 0, 1, 4, 3])

# Remap the edge_index
remapped_edge_index = remap_edge_index(edge_index, node_mapping)
print(remapped_edge_index)


tensor([[2, 0, 1, 4],
        [0, 1, 4, 3]])


In [11]:
import torch

def remove_edges(orig_edge_index, edge_index):
    # Function to convert [2, num_edges] to unique identifier format for each edge
    def edge_to_tuple_tensor(edge_index):
        return torch.cat((edge_index[0].unsqueeze(1), edge_index[1].unsqueeze(1)), dim=1)

    # Convert edge indices to tuple format
    orig_edges = edge_to_tuple_tensor(orig_edge_index)
    remove_edges = edge_to_tuple_tensor(edge_index)

    # Use a unique representation of each edge for easy comparison
    # Convert each pair to a large number (assuming max node index is reasonably small)
    max_node = max(orig_edge_index.max(), edge_index.max()) + 1
    orig_edges_flat = orig_edges[:, 0] * max_node + orig_edges[:, 1]
    remove_edges_flat = remove_edges[:, 0] * max_node + remove_edges[:, 1]

    # Create a mask for original edges not in remove_edges
    remaining_edges_mask = ~(orig_edges_flat.unsqueeze(1) == remove_edges_flat).any(dim=1)

    # Filter the original edges using the mask
    remaining_edges = orig_edge_index[:, remaining_edges_mask]

    return remaining_edges

# Example usage
orig_edge_index = torch.tensor([[0, 1, 2, 3, 4],
                                [1, 2, 3, 4, 0]])
edge_index = torch.tensor([[0, 1, 4],
                           [1, 2, 0]])

# Remove the edges specified by edge_index from orig_edge_index
remaining_edge_index = remove_edges(orig_edge_index, edge_index)
print(remaining_edge_index)


tensor([[2, 3],
        [3, 4]])


In [15]:
import torch

def assert_edges_exist(orig_edge_index, edge_index):
    # Function to convert [2, num_edges] to a unique identifier for each edge
    def edge_to_unique_id(edge_index):
        max_node = max(orig_edge_index.max(), edge_index.max()) + 1
        return edge_index[0] * max_node + edge_index[1]

    # Convert edge indices to unique identifiers
    orig_edge_ids = edge_to_unique_id(orig_edge_index)
    edge_ids = edge_to_unique_id(edge_index)

    # Check for each edge in edge_index if it exists in orig_edge_index
    exists = torch.isin(edge_ids, orig_edge_ids)

    # Assert all edges in edge_index exist in orig_edge_index
    assert exists.all(), "Some edges in edge_index do not exist in orig_edge_index."

# Example usage
orig_edge_index = torch.tensor([[0, 1, 2, 3, 4],
                                [1, 2, 3, 4, 0]])
edge_index = torch.tensor([[0, 1],
                           [1, 2]])

# Run the assertion to ensure all edges in edge_index exist in orig_edge_index
assert_edges_exist(orig_edge_index, edge_index)  # Should pass without error

# An example where the assertion will fail
test_edge_index = torch.tensor([[0, 2],  # Edge (0,5) does not exist in orig_edge_index
                                [1, 3]])
try:
    assert_edges_exist(orig_edge_index, test_edge_index)
except AssertionError as e:
    print(e)  # Output the assertion error message


In [10]:
import torch
from torch_geometric.utils import k_hop_subgraph

# Example data
node_idx = 0
edge_index = torch.tensor([[1, 1, 2, 3, 4, 0, 1, 3, 4, 0],
                           [0, 2, 3, 4, 0, 2, 3, 4, 0, 1]])
x = torch.randn((5, 2))  # Example node features
pyg_data = Data(x=x, edge_index=edge_index)

# Call k_hop_subgraph to get the 2-hop neighborhood
hop2_subset, hop2_edge_index, hop2_mapping, hop2_edge_mask = k_hop_subgraph(
    node_idx, 2, pyg_data.edge_index, relabel_nodes=True, flow='source_to_target')

# Create the mapping from relabeled node_id to original one
relabeled_to_original = {i: hop2_subset[i].item() for i in range(len(hop2_subset))}

# Print the mapping
print(hop2_subset)
print("Relabeled to Original Node Mapping:")
for relabeled_id, original_id in relabeled_to_original.items():
    print(f"Relabeled ID {relabeled_id} -> Original ID {original_id}")


tensor([0, 1, 3, 4])
Relabeled to Original Node Mapping:
Relabeled ID 0 -> Original ID 0
Relabeled ID 1 -> Original ID 1
Relabeled ID 2 -> Original ID 3
Relabeled ID 3 -> Original ID 4


In [6]:
data = GADDataset('reddit')

In [16]:
data.graph

Graph(num_nodes=10984, num_edges=168016,
      ndata_schemes={'feature': Scheme(shape=(64,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_masks': Scheme(shape=(20,), dtype=torch.uint8), 'val_masks': Scheme(shape=(20,), dtype=torch.uint8), 'test_masks': Scheme(shape=(20,), dtype=torch.uint8)}
      edata_schemes={'count': Scheme(shape=(), dtype=torch.int64)})

In [17]:
data.graph.edata['count']

tensor([2, 1, 1,  ..., 2, 1, 2])

In [18]:
aug_dataset = torch.load('/home/allenwang/gad_diff/gad_lggm/outputs/aug_reddit_onehot/augment_samples.pt')

In [27]:
pyg_data = aug_dataset[0]
src, dst = pyg_data.edge_index

dgl_graph = dgl.graph((src, dst))
dgl_graph.ndata['label'] = torch.argmax(pyg_data.x, dim=1)
dgl_graph.ndata['label']

tensor([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0])

In [12]:
raw_reddit = torch.load('pyg_dataset/reddit.pt')
x = raw_reddit.x
y = raw_reddit.y

torch.cat([x, y.unsqueeze(1)], dim=1)

tensor([[0.0168, 0.0151, 0.0158,  ..., 0.0153, 0.0222, 0.0000],
        [0.0160, 0.0154, 0.0152,  ..., 0.0153, 0.0169, 0.0000],
        [0.0151, 0.0160, 0.0169,  ..., 0.0156, 0.0174, 0.0000],
        ...,
        [0.0175, 0.0158, 0.0170,  ..., 0.0156, 0.0189, 0.0000],
        [0.0153, 0.0152, 0.0152,  ..., 0.0156, 0.0157, 0.0000],
        [0.0064, 0.0133, 0.0124,  ..., 0.0157, 0.0166, 0.0000]])

In [2]:
reddit = torch.load('pyg_dataset/reddit/reddit_v2.pt')
reddit[0]

(tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 1., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 1., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]]),
 tensor([[0.0114, 0.0165, 0.0133,  ..., 0.0156, 0.0178, 0.0000],
         [0.0209, 0.0172, 0.0190,  ..., 0.0156, 0.0174, 0.0000],
         [0.0124, 0.0109, 0.0177,  ..., 0.0158, 0.0169, 0.0000],
         ...,
         [0.0176, 0.0173, 0.0175,  ..., 0.0154, 0.0175, 0.0000],
         [0.0122, 0.0143, 0.0132,  ..., 0.0154, 0.0173, 0.0000],
         [0.0151, 0.0160, 0.0169,  ..., 0.0156, 0.0174, 0.0000]]))

In [2]:
def get_khop_subgraph(pyg_data, node_idx, maxN):
    # Extract 2-hop subgraph
    hop2_subset, hop2_edge_index, hop2_mapping, hop2_edge_mask = k_hop_subgraph(
        node_idx, 2, pyg_data.edge_index, relabel_nodes=True, flow='source_to_target')

    # Convert the 2-hop subgraph to undirected
    hop2_edge_index = to_undirected(hop2_edge_index)

    # Perform random walk to sample nodes until maxN unique nodes are reached
    if len(hop2_subset) > maxN:
        walk_start = hop2_mapping[0].item() # center node
        walks = random_walk_until_maxN(hop2_edge_index[0], hop2_edge_index[1], torch.tensor([walk_start]), maxN, walk_length=2)
        subsample_subset = torch.unique(walks.flatten())
        
        while len(subsample_subset) < maxN:
            # keep random walking until we have maxN unique nodes
            walks = random_walk_until_maxN(hop2_edge_index[0], hop2_edge_index[1], torch.tensor([walk_start]), maxN, walk_length=2)
            # concatenate the new walks with the previous walks
            subsample_subset = torch.unique(torch.cat((subsample_subset, torch.unique(walks.flatten()))))

        if len(subsample_subset) > maxN:
            subsample_subset = subsample_subset[:maxN]

        subsample_edge_index, subsample_edge_mask = subgraph(subsample_subset, hop2_edge_index, relabel_nodes=True)
    else:
        subsample_subset = hop2_subset
        subsample_edge_index = hop2_edge_index

    # Create subgraph data object
    subgraph_data = Data(x=pyg_data.x[subsample_subset], edge_index=subsample_edge_index, num_nodes=len(subsample_subset))

    # Set the label for the subgraph based on the original node's label
    label = pyg_data.y[node_idx].item()
    subgraph_data.y = torch.tensor([label], dtype=torch.long)
    subgraph_data.center_node_idx = node_idx

    return subgraph_data

In [15]:
pyg_data = torch.load('./pyg_dataset/reddit.pt')
pyg_data 

Data(edge_index=[2, 168016], train_masks=[10984], val_masks=[10984], test_masks=[10984], num_nodes=10984, y=[10984], x=[10984, 64])

In [17]:
train_indices = torch.nonzero(pyg_data.train_masks, as_tuple=False).squeeze().tolist()
vali_indices = torch.nonzero(pyg_data.val_masks, as_tuple=False).squeeze().tolist()
test_indices = torch.nonzero(pyg_data.test_masks, as_tuple=False).squeeze().tolist()

print(len(train_indices), len(vali_indices), len(test_indices))

4393 2196 4395


In [18]:
# examine the clustering coefficient of the 2-hop subgraph of training nodes with k_hop_subgraph from PyG
import networkx as nx
from torch_geometric.utils import to_networkx, to_undirected, k_hop_subgraph
from tqdm import tqdm
import numpy as np


train_clustering_coefficients = []

for node_idx in tqdm(vali_indices):
    subset, edge_index, mapping, edge_mask = k_hop_subgraph(node_idx, 1, pyg_data.edge_index, relabel_nodes=True, flow='source_to_target')
    edge_index = to_undirected(edge_index)
    G = to_networkx(Data(edge_index=edge_index))
    clustering_coefficient = nx.average_clustering(G)
    train_clustering_coefficients.append(clustering_coefficient)

print(f"Average clustering coefficient of 2-hop subgraph of training nodes: {np.mean(train_clustering_coefficients)}")

100%|██████████| 2196/2196 [00:02<00:00, 1015.83it/s]

Average clustering coefficient of 2-hop subgraph of training nodes: 0.0


In [53]:
anomaly_idx = (pyg_data.y == 1).nonzero().squeeze()
train_anomaly_idx = list(set(anomaly_idx.tolist()).intersection(set(train_indices)))
train_anomaly_idx[:20]

[0,
 4,
 8,
 9,
 8204,
 19,
 8230,
 8234,
 51,
 52,
 8250,
 8251,
 58,
 59,
 62,
 8260,
 8265,
 74,
 75,
 76]

In [58]:
subgraph_list = []  
for idx in train_anomaly_idx:
    subgraph_data = get_khop_subgraph(pyg_data, idx, 100)
    subgraph_list.append(subgraph_data)
    print(subgraph_data)

Data(x=[1, 10], edge_index=[2, 1], num_nodes=1, y=[1], center_node_idx=0)
Data(x=[2, 10], edge_index=[2, 4], num_nodes=2, y=[1], center_node_idx=4)
Data(x=[1, 10], edge_index=[2, 1], num_nodes=1, y=[1], center_node_idx=8)
Data(x=[1, 10], edge_index=[2, 1], num_nodes=1, y=[1], center_node_idx=9)
Data(x=[19, 10], edge_index=[2, 139], num_nodes=19, y=[1], center_node_idx=8204)
Data(x=[1, 10], edge_index=[2, 1], num_nodes=1, y=[1], center_node_idx=19)
Data(x=[88, 10], edge_index=[2, 1672], num_nodes=88, y=[1], center_node_idx=8230)
Data(x=[100, 10], edge_index=[2, 2154], num_nodes=100, y=[1], center_node_idx=8234)
Data(x=[1, 10], edge_index=[2, 1], num_nodes=1, y=[1], center_node_idx=51)
Data(x=[11, 10], edge_index=[2, 43], num_nodes=11, y=[1], center_node_idx=52)
Data(x=[100, 10], edge_index=[2, 2076], num_nodes=100, y=[1], center_node_idx=8250)
Data(x=[100, 10], edge_index=[2, 4082], num_nodes=100, y=[1], center_node_idx=8251)
Data(x=[1, 10], edge_index=[2, 1], num_nodes=1, y=[1], center

In [68]:
subgraph_data

Data(x=[2, 10], edge_index=[2, 4], num_nodes=2, y=[1], center_node_idx=1)

In [53]:
len(subset)

10109

In [40]:
import networkx as nx
nx_graph = to_networkx(subgraph, to_undirected=True)
components = list(nx.connected_components(nx_graph))
if len(components) > 1:
    print(f"Graph contains {len(components)} connected components.")
else:
    print("Graph is connected.")

Graph is connected.


In [62]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import k_hop_subgraph

# Create a sample graph
edge_index = torch.tensor([[0, 1, 2, 3, 4, 5],
                           [2, 2, 4, 4, 6, 6]], dtype=torch.long)

# Parameters
node_idx = 6  # Node to start the k-hop subgraph extraction
num_hops = 2  # Number of hops

# Extract k-hop subgraph (source to target)
subset_s2t, edge_index_s2t, _, _ = k_hop_subgraph(node_idx, num_hops, edge_index, flow='source_to_target')

# Extract k-hop subgraph (target to source)
subset_t2s, edge_index_t2s, _, _ = k_hop_subgraph(node_idx, num_hops, edge_index, flow='target_to_source')

# Print results
print("Original Edge Index:")
print(edge_index)

print("\nSubset nodes (source to target):")
print(subset_s2t)

print("Edge Index (source to target):")
print(edge_index_s2t)

print("\nSubset nodes (target to source):")
print(subset_t2s)

print("Edge Index (target to source):")
print(edge_index_t2s)


Original Edge Index:
tensor([[0, 1, 2, 3, 4, 5],
        [2, 2, 4, 4, 6, 6]])

Subset nodes (source to target):
tensor([2, 3, 4, 5, 6])
Edge Index (source to target):
tensor([[2, 3, 4, 5],
        [4, 4, 6, 6]])

Subset nodes (target to source):
tensor([6])
Edge Index (target to source):
tensor([], size=(2, 0), dtype=torch.int64)


## RootedRWSampling

In [10]:
from utils import *
import torch

In [11]:
data = torch.load('./pyg_dataset/tolokers.pt')
data

Data(edge_index=[2, 530758], train_masks=[11758], val_masks=[11758], test_masks=[11758], num_nodes=11758, y=[11758], x=[11758, 10])

In [14]:
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected, k_hop_subgraph, subgraph
from torch_cluster import random_walk
import torch

class RootedRWSubgraph(RootedSubgraph):
    def __init__(self, walk_length: int, repeat: int = 1):
        super().__init__()
        self.walk_length = walk_length
        self.repeat = repeat

    def extract(
        self,
        data: Data,
        node_id: int,
    ) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor]:
        from torch_cluster import random_walk

        assert data.edge_index is not None
        num_nodes = data.num_nodes
        assert num_nodes is not None

        start = torch.tensor([node_id], device=data.edge_index.device)
        start = start.view(-1, 1).repeat(1, self.repeat).view(-1)
        walk = random_walk(data.edge_index[0], data.edge_index[1], start,
                           self.walk_length, num_nodes=data.num_nodes)

        n_mask = torch.zeros((num_nodes, num_nodes), dtype=torch.bool,
                             device=walk.device)
        start = start.view(-1, 1).repeat(1, (self.walk_length + 1)).view(-1)
        n_mask[start, walk.view(-1)] = True

        return self.map(data, n_mask)

    def forward(self, data: Data, node_ids: List[int]) -> List[Data]:
        subgraphs = []
        for node_id in node_ids:
            out = self.extract(data, node_id)
            d = RootedSubgraphData.from_dict(data.to_dict())
            d.sub_edge_index, d.n_id, d.e_id, d.n_sub_batch, d.e_sub_batch = out
            subgraphs.append(d.map_data())
        return subgraphs
    
def get_random_walk_subgraph(pyg_data, node_idx, walk_length=6, repeat=1):
    transform = RootedRWSubgraph(walk_length=walk_length, repeat=repeat)
    subgraph_data_list = transform.forward(pyg_data, [node_idx])
    
    # Assuming we need only the first (and only) subgraph in the list
    subgraph_data = subgraph_data_list[0]

    # Set the label for the subgraph based on the original node's label
    label = pyg_data.y[node_idx].item()
    subgraph_data.y = torch.tensor([label], dtype=torch.long)
    subgraph_data.center_node_idx = node_idx

    return subgraph_data

In [18]:
node_idx = 0  # Node index from which to start the random walk
subgraph_data = get_random_walk_subgraph(data, node_idx, walk_length=150, repeat=1)

In [19]:
subgraph_data

RootedSubgraphData(x=[7, 10], edge_index=[2, 17], y=[1], train_masks=[7], val_masks=[7], test_masks=[7], num_nodes=7, n_sub_batch=[7], center_node_idx=0)

## Diffusion Model

In [1]:
import torch

reddit = torch.load('pyg_dataset/reddit/reddit_continuous.pt')

In [3]:
len(reddit[0])

3

In [12]:
a,x = reddit[0]

ValueError: too many values to unpack (expected 2)

In [ ]:
if data.has_attr('attribute_name'):
    print("Attribute exists")
else:
    print("Attribute does not exist")